In [1]:
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize

In [2]:
def extract_ne_from_tree ( tree ):
    result = []
    for s in tree.subtrees():
        label = s.label()
        if (label == 'PERSON' or label == 'GPE' or label == 'LOCATION'):
            leaves = s.leaves()
            ne = ''
            for l in leaves:
                ne = ne + ' ' + l[0]
            result.append((label, ne[1:]))
    return result

### cnn news

In [7]:
# merge cnn news into 1 dataframe
# cnn_27feb = pd.read_csv('cnn_27feb.csv')
# cnn_28feb = pd.read_csv('cnn_28feb.csv')
# cnn_1mar = pd.read_csv('cnn_1mar.csv')
# cnn_2mar = pd.read_csv('cnn_2mar.csv')
# cnn_3mar = pd.read_csv('cnn_3mar.csv')
# cnn_8mar = pd.read_csv('cnn_8mar.csv')
# cnn_9mar = pd.read_csv('cnn_9mar.csv')
# cnn = pd.concat([cnn_27feb, cnn_28feb, cnn_1mar, cnn_2mar, cnn_3mar, cnn_8mar, cnn_9mar])
# print(cnn.shape)
# cnn.head()

import glob
import os

path = r'C:\latest_news_scraper\cna_news' # use your path
all_files = glob.glob(path + "/*.csv")

df_list = (pd.read_csv(file) for file in all_files)
cnn   = pd.concat(df_list, ignore_index=True)
cnn = cnn.loc[:,['url','content']]
cnn

,url,content
0,https://www.channelnewsasia.com/sport/joseph-s...,SINGAPORE: Joseph Schooling on Wednesday (Mar ...
1,https://www.channelnewsasia.com/sport/hsbc-wom...,Lydia Ko and Nelly Korda headline a stellar fi...
2,https://www.channelnewsasia.com/sport/novak-dj...,Novak Djokovic said his return to the top of t...
3,https://www.channelnewsasia.com/sport/djokovic...,Novak Djokovic won on his return to the ATP To...
4,https://www.channelnewsasia.com/sport/top-seed...,Top seed Carlos Alcaraz has pulled out of this...
...,...,...
140,https://www.channelnewsasia.com/sport/bayern-f...,"Bayern Munich have been fined 44,750 euros ($4..."
141,https://www.channelnewsasia.com/sport/bbcs-lin...,Gary Lineker was never booked in his 16-year p...
142,https://www.channelnewsasia.com/sport/ive-been...,Amid reports of opposition from players and un...
143,https://www.channelnewsasia.com/sport/world-sp...,"MANCHESTER, England: FIFA General Secretary Fa..."


In [4]:
#cnn_content = cnn[['content']]
#cnn_content = cnn_content.replace('\n',' ', regex=True)
#print(cnn_content)

In [8]:
cnn_content = []
for x in cnn['content']:
    cnn_sent = nltk.sent_tokenize(x)
    cnn_tokens = [word_tokenize(sents) for sents in cnn_sent]
    tags = [nltk.pos_tag(tokens) for tokens in cnn_tokens]
    cnn_tree = [nltk.ne_chunk(elem) for elem in tags]
    cnn_list = [extract_ne_from_tree(tree) for tree in cnn_tree]
    cnn_content+=[(cnn_list)]
    
print(cnn_content)

[[[('GPE', 'SINGAPORE'), ('LOCATION', 'Southeast')], [], [], [], [], [], [('GPE', 'Cambodia')]], [[('GPE', 'Lydia'), ('PERSON', 'Nelly Korda'), ('GPE', 'Singapore')], [('GPE', 'New'), ('PERSON', 'Zealander Ko'), ('GPE', 'Saudi Arabia'), ('GPE', 'Singapore')], [('PERSON', 'Ko')], [], [], [('GPE', 'Australian'), ('PERSON', 'Ko Jin-young')], [('LOCATION', 'South Korean'), ('GPE', 'August')], [('GPE', 'Thailand')], [], [], [('GPE', 'South'), ('GPE', 'Asia')], [], [('PERSON', 'Park In-bee')]], [[('PERSON', 'Novak'), ('PERSON', 'Djokovic'), ('GPE', 'Serbian')], [('PERSON', 'Djokovic')], [('GPE', 'United States')], [('PERSON', 'Grand Slam'), ('GPE', 'Russia'), ('GPE', 'Belarus'), ('GPE', 'Moscow'), ('GPE', 'Ukraine')], [], [('PERSON', 'Djokovic'), ('PERSON', 'Tomas Machac'), ('GPE', 'Dubai')], [('GPE', 'Grand Slams')], [], [], [], [], [], [('GPE', 'Djokovic'), ('PERSON', 'Steffi Graf')], [('PERSON', 'Carlos Alcaraz'), ('PERSON', 'Stefanos Tsitsipas')], [], [], [('PERSON', 'Djokovic')], [], []

In [9]:
location = []
gpe = []
person = []

for row in cnn_content:
    check_location = []
    check_gpe = []
    check_person = []
    for sent in row:
        if len(sent)!=0:
            for tup in sent:
                if tup[0] == "LOCATION":
                    if tup[1] not in check_location:
                        check_location.append(tup[1])
                if tup[0] == "GPE":
                    if tup[1] not in check_gpe:
                        check_gpe.append(tup[1])
                if tup[0] == "PERSON":
                    if tup[1] not in check_person:
                        check_person.append(tup[1])
    location.append(check_location)
    gpe.append(check_gpe)
    person.append(check_person)

print(location)
print()

print(gpe)
print()

print(person)
print()

[['Southeast'], ['South Korean'], [], [], [], [], ['West Ham United'], [], [], [], [], [], [], [], [], [], [], ['Sebastian Munoz'], [], [], [], [], [], [], [], [], [], [], [], [], ['West Ham United'], [], ['South Africa', 'Caribbean'], [], [], [], ['West'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['West Ham United'], [], ['South Africa', 'Caribbean'], [], [], [], [], [], [], ['West Germany', 'West German'], ['West'], ['West Ham'], [], ['Western Force'], [], [], [], [], [], [], ['West Ham'], [], [], [], [], [], [], [], ['West', 'West Indies'], [], [], [], [], [], [], [], [], ['South'], [], [], [], [], [], [], [], [], [], ['West Ham United', 'West Bromwich'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['Northern Irishman'], [], ['Southern California'], [], [], [], [], [], ['East Brabant'], [], [], [], [], []]

[['SINGAPORE', 'Cambodia'], ['Lydia', 'Singapore', 'New', 'Saudi Arabia', 'Aust

In [10]:
cnn['location'] = location
cnn['gpe'] = gpe
cnn['person'] = person

cnn.head()

,url,content,location,gpe,person
0,https://www.channelnewsasia.com/sport/joseph-s...,SINGAPORE: Joseph Schooling on Wednesday (Mar ...,[Southeast],"[SINGAPORE, Cambodia]",[]
1,https://www.channelnewsasia.com/sport/hsbc-wom...,Lydia Ko and Nelly Korda headline a stellar fi...,[South Korean],"[Lydia, Singapore, New, Saudi Arabia, Australi...","[Nelly Korda, Zealander Ko, Ko, Ko Jin-young, ..."
2,https://www.channelnewsasia.com/sport/novak-dj...,Novak Djokovic said his return to the top of t...,[],"[Serbian, United States, Russia, Belarus, Mosc...","[Novak, Djokovic, Grand Slam, Tomas Machac, St..."
3,https://www.channelnewsasia.com/sport/djokovic...,Novak Djokovic won on his return to the ATP To...,[],"[Australian, Djokovic, Netherlands, Doha, Ital...","[Novak, Djokovic, Czech, Tomas Machac, Dubai T..."
4,https://www.channelnewsasia.com/sport/top-seed...,Top seed Carlos Alcaraz has pulled out of this...,[],"[Acapulco, U.S., Spanish, Norrie]","[Carlos Alcaraz, Cameron Norrie, Alcaraz, Span..."


In [18]:
location_remove_brackets=cnn['location'].apply(lambda x: ','.join(map(str, x)))
gpe_remove_brackets = cnn['gpe'].apply(lambda x: ', '.join(map(str, x)))
person_remove_brackets = cnn['person'].apply(lambda x: ', '.join(map(str, x)))
person_remove_brackets

0                                                       
1      Nelly Korda, Zealander Ko, Ko, Ko Jin-young, P...
2      Novak, Djokovic, Grand Slam, Tomas Machac, Ste...
3      Novak, Djokovic, Czech, Tomas Machac, Dubai Te...
4      Carlos Alcaraz, Cameron Norrie, Alcaraz, Spaniard
                             ...                        
140                                     Paris St Germain
141    Gary, Lineker, Match, Brexit, Suella Braverman...
142    Corinne Diacre, Wendie Renard, Renard, Diacre,...
143    Fatma Samoura, Morocco, Vietnam, Zambia, Samou...
144    Club Brugge, Scott Parker, Benfica, Brugge, Fu...
Name: person, Length: 145, dtype: object

### cna news

In [6]:
# Merging cna news into 1 dataframe
cna_27feb = pd.read_csv('cna_27feb.csv')
cna_28feb = pd.read_csv('cna_28feb.csv')
cna_1mar = pd.read_csv('cna_1mar.csv')
cna_2mar = pd.read_csv('cna_2mar.csv')
cna_3mar = pd.read_csv('cna_3mar.csv')
cna_8mar = pd.read_csv('cna_8mar.csv')
cna_9mar = pd.read_csv('cna_9mar.csv')
cna = pd.concat([cna_27feb, cna_28feb, cna_1mar, cna_2mar, cna_3mar, cna_8mar, cna_9mar])
print(cna.shape)
cna.head()

(145, 3)


,Unnamed: 0,url,content
0,0,https://www.channelnewsasia.com/sport/djokovic...,Novak Djokovic said he was now playing without...
1,2,https://www.channelnewsasia.com/sport/future-l...,LONDON: Manchester United's near six-year trop...
2,4,https://www.channelnewsasia.com/sport/clinical...,LONDON: Manchester United crushed Newcastle Un...
3,6,https://www.channelnewsasia.com/sport/klinsman...,Juergen Klinsmann has been named head coach of...
4,7,https://www.channelnewsasia.com/sport/spurs-be...,LONDON: Tottenham Hotspur beat Chelsea 2-0 in ...
